In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/customer_churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [3]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
encoder = LabelEncoder()
df['Gender'] = encoder.fit_transform(df['Gender'])


In [5]:
df['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [6]:
one_hot = OneHotEncoder(sparse_output=False)
df_geo = one_hot.fit_transform(df[['Geography']])
df_final = pd.DataFrame(df_geo,columns = one_hot.get_feature_names_out(['Geography']))
df_final

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
df = pd.concat([df,df_final],axis=1)

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
df = df.drop('Geography',axis=1)

In [10]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
X = df.drop("Exited",axis=1)
y = df['Exited']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
#ANN IMPLIMENTED
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
#log_dir = "table/fit" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
#tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
#early_stopping_callback = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [16]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,verbose=1)

Epoch 1/10


250/250 [==============================] - 8s 8ms/step - loss: 0.4486 - accuracy: 0.8100 - val_loss: 0.3966 - val_accuracy: 0.8320
Epoch 2/10
250/250 [==============================] - 1s 5ms/step - loss: 0.3889 - accuracy: 0.8409 - val_loss: 0.3668 - val_accuracy: 0.8450
Epoch 3/10
250/250 [==============================] - 1s 5ms/step - loss: 0.3579 - accuracy: 0.8531 - val_loss: 0.3492 - val_accuracy: 0.8595
Epoch 4/10
250/250 [==============================] - 1s 5ms/step - loss: 0.3431 - accuracy: 0.8590 - val_loss: 0.3434 - val_accuracy: 0.8630
Epoch 5/10
250/250 [==============================] - 2s 8ms/step - loss: 0.3372 - accuracy: 0.8599 - val_loss: 0.3428 - val_accuracy: 0.8615
Epoch 6/10
250/250 [==============================] - 2s 9ms/step - loss: 0.3341 - accuracy: 0.8606 - val_loss: 0.3417 - val_accuracy: 0.8580
Epoch 7/10
250/250 [==============================] - 2s 7ms/step - loss: 0.3310 - accuracy: 0.8625 - val_loss: 0.3430 - val_accuracy: 0.8585
Epoc

In [17]:
y_pred = model.predict(X_test)
y_result = (y_pred > 0.5).astype('int32')
print(y_result[:5])

63/63 [==============================] - 0s 3ms/step
[[0]
 [0]
 [0]
 [0]
 [0]]


In [18]:
new_data = {
    'CreditScore': 600,
    'Gender': 1, 
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Geography_France': 1,
    'Geography_Germany': 0,
    'Geography_Spain': 0
}


new_df = pd.DataFrame([new_data])
new_data_scaled = scaler.transform(new_df)


prediction_prob = model.predict(new_data_scaled)
prob = prediction_prob[0][0]


print(f"\nChances: {prob * 100:.2f}%")

if prob > 0.5:
    print("Result: Yes (Customer leave karega)")
else:
    print("Result: No (Customer nahi jayega)")

1/1 [==============================] - 0s 75ms/step

Chances: 2.96%
Result: No (Customer nahi jayega)
